In [1]:
# Generalized Estimating Equations
# https://github.com/statsmodels/statsmodels/wiki/Examples#generalized-estimating-equations-gee

In [ ]:
# The following illustrates a Poisson regression with exchangeable correlation within clusters using data on epilepsy seizures.

In [3]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [4]:
data = sm.datasets.get_rdataset('epil', package='MASS').data
fam = sm.families.Poisson()
ind = sm.cov_struct.Exchangeable()

In [5]:
mod = smf.gee("y ~ age + trt + base", "subject", data, cov_struct=ind, family=fam)

In [6]:
res = mod.fit()

In [7]:
print(res.summary())

                               GEE Regression Results                              
Dep. Variable:                           y   No. Observations:                  236
Model:                                 GEE   No. clusters:                       59
Method:                        Generalized   Min. cluster size:                   4
                      Estimating Equations   Max. cluster size:                   4
Family:                            Poisson   Mean cluster size:                 4.0
Dependence structure:         Exchangeable   Num. iterations:                    51
Date:                     Thu, 14 Feb 2019   Scale:                           1.000
Covariance type:                    robust   Time:                         13:58:14
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.5730      0.361      1.589      0.112      -0.134  

In [8]:
# Key ideas: Nested dependence structures
# https://nbviewer.jupyter.org/urls/umich.box.com/shared/static/wt4jlup9nwbt2d69xvm6.ipynb?create=1

In [9]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [10]:
# The data file has no column labels, so we attach them manually.

In [11]:
data = pd.read_csv("Exam.txt", header=None, sep=' ')
data.columns = ["Board", "A-Score", "Gtot", "Gnum",
                "Gender", "Age", "Inst_Type", "LEA",
                "Institute", "Student"]

In [12]:
data.head()

,Board,A-Score,Gtot,Gnum,Gender,Age,Inst_Type,LEA,Institute,Student
0,1.0,143.0,0.261324,1,0.619059,1,1,0.166175,2,1
1,1.0,145.0,0.134067,1,0.205802,1,1,0.166175,2,2
2,1.0,142.0,-1.723882,1,-1.364576,0,1,0.166175,2,3
3,1.0,141.0,0.967586,1,0.205802,1,1,0.166175,2,2
4,1.0,138.0,0.544341,1,0.371105,1,1,0.166175,2,2


In [ ]:
# The variable 'Gtot' in the data set is the sum of exam scores over all the components taken by a given student. 
# We divide the score sum by the number of components taken, in order to obtain a mean score that is not inflated just because 
# a student takes more exams.

In [13]:
data["GCSE"] = data["Gtot"] / data["Gnum"]

In [14]:
# we need to create a new variable that combines the two codes.
data["School"] = [str(x) + str(y) for x,y in zip(data["LEA"], data["Institute"])]
us = set(data["School"])
us = {x: k for k,x in enumerate(list(us))}
data["School"] = [us[x] for x in data["School"]]

In [ ]:
# These are all the variables in the analysis.

In [15]:
qdata = data[["GCSE", "Gender", "Age", "LEA", "Institute", "School"]]
qdata = np.asarray(qdata)

In [18]:
family = sm.families.Gaussian()

In [19]:
# The initial model considers only the school effect, and does not account for the fact that the schools are nested in LEA's.

In [21]:
ex = sm.cov_struct.Exchangeable()
model1 = sm.GEE.from_formula("GCSE ~ Age + Gender", "LEA",
                       data=data, family=family, cov_struct=ex)
result1 = model1.fit()
print (result1.summary())

                               GEE Regression Results                              
Dep. Variable:                        GCSE   No. Observations:                 4059
Model:                                 GEE   No. clusters:                       65
Method:                        Generalized   Min. cluster size:                   2
                      Estimating Equations   Max. cluster size:                 198
Family:                           Gaussian   Mean cluster size:                62.4
Dependence structure:         Exchangeable   Num. iterations:                     5
Date:                     Thu, 14 Feb 2019   Scale:                           0.643
Covariance type:                    robust   Time:                         14:19:39
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0942      0.040     -2.349      0.019      -0.173      -0.016
Age    

In [23]:
# The dependence among subjects within a school is fairly weak:
print (ex.summary())

The correlation between two observations in the same cluster is 0.109


In [25]:
# Next we fit the model using the nested dependence structure. Since the iterations are somewhat slow, 
# we limit the number of iterations. This may result in a warning message.
ne = sm.cov_struct.Nested()
model2 = sm.GEE.from_formula("GCSE ~ Age + Gender", "LEA",
                        data=data, family=family, cov_struct=ne,
                        dep_data=data["Institute"])
result2 = model2.fit() #maxiter=10)
print (result2.summary())

                               GEE Regression Results                              
Dep. Variable:                        GCSE   No. Observations:                 4059
Model:                                 GEE   No. clusters:                       65
Method:                        Generalized   Min. cluster size:                   2
                      Estimating Equations   Max. cluster size:                 198
Family:                           Gaussian   Mean cluster size:                62.4
Dependence structure:               Nested   Num. iterations:                    60
Date:                     Thu, 14 Feb 2019   Scale:                           0.725
Covariance type:                    robust   Time:                         14:22:24
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.1967      0.147      1.335      0.182      -0.092       0.485
Age    

C:\Users\kxi110617\AppData\Local\conda\conda\envs\deeplearning\lib\site-packages\statsmodels\genmod\generalized_estimating_equations.py:1128: IterationLimitWarning: Iteration limit reached prior to convergence
  IterationLimitWarning)


In [27]:
print (ne.summary())

Variance estimates
------------------
Component 1: 0.000
Component 2: 24013633618002.078
Residual: -24013633618001.352



In [ ]:
# https://nbviewer.jupyter.org/urls/umich.box.com/shared/static/wt4jlup9nwbt2d69xvm6.ipynb?create=1